In [16]:
import pandas as pd
df = pd.read_csv('eapv_21032022_13h29.csv', sep=';')
print(f'Banco importado: {df.shape}')
banco_completo = df.shape[0]

Banco importado: (18184, 189)


/var/folders/4y/plk7ct8n4g9gz40s3_dvz7cm0000gn/T/ipykernel_17841/3906717128.py:2: DtypeWarning: Columns (22,34,52,63,64,75,122,137,152,160,161,162,163,164,165,166,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eapv_21032022_13h29.csv', sep=';')


In [17]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_21032022.csv', sep=';', encoding='utf-8')
# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
print(f'Banco canceladas: {df_canceladas.shape}')
banco_canceladas = df_canceladas.shape[0]
# Transformando o banco df_canceladas para uma lista notif_canceladas
notif_canceladas = df_canceladas['Número da Notificação'].to_list()

Banco canceladas: (1002, 1)


In [18]:
# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]

# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')

# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

Banco sem as canceladas: 17182
Banco completo: 18184 - canceladas: 1002 = 17182


In [19]:
# Encerrados
df_encerrados = df.copy()
df_encerrados['Encerramento'] = 'Nao encerrado'

# Municipio
df_encerrados['Municipal'] = 'Municipal'
df_encerrados['Encerramento Municipal - Evento adverso'].fillna('Not', inplace=True)
df_encerrados.loc[df_encerrados['Encerramento Municipal - Evento adverso'].str.contains('Not', na = False), 'Municipal'] = 'Nao encerrado'

# Estadual
df_encerrados['Estadual'] = 'Estadual'
df_encerrados['Encerramento Estadual - Evento adverso'].fillna('Not estadual', inplace=True)
df_encerrados.loc[df_encerrados['Encerramento Estadual - Evento adverso'].str.contains('Not estadual', na = False), 'Estadual'] = 'Nao encerrado'

# Federal 
df_encerrados['Federal'] = 'Federal'
df_encerrados['Encerramento Federal - Evento adverso'].fillna('Not federal', inplace=True)
df_encerrados.loc[df_encerrados['Encerramento Federal - Evento adverso'].str.contains('Not federal', na = False), 'Federal'] = 'Nao encerrado'

# Federal 
df_encerrados['Cifavi'] = 'Cifavi'
df_encerrados['Encerramento Cifavi - Evento adverso'].fillna('Not cifavi', inplace=True)
df_encerrados.loc[df_encerrados['Encerramento Cifavi - Evento adverso'].str.contains('Not cifavi', na = False), 'Cifavi'] = 'Nao encerrado'

In [20]:
df_encerrados.loc[df_encerrados['Municipal'].str.contains('Municipal', na = False), 'Encerramento'] = 'Municipal'
df_encerrados.loc[df_encerrados['Estadual'].str.contains('Estadual', na = False), 'Encerramento'] = 'Estadual'
df_encerrados.loc[df_encerrados['Federal'].str.contains('Federal', na = False), 'Encerramento'] = 'Federal'

df_encerrados = df_encerrados.drop(columns=['Municipal'])
df_encerrados = df_encerrados.drop(columns=['Estadual'])
df_encerrados = df_encerrados.drop(columns=['Federal'])
df_encerrados = df_encerrados.drop(columns=['Cifavi'])

In [21]:
df_encerrados['Encerramento'].unique()

array(['Municipal', 'Estadual', 'Nao encerrado', 'Federal'], dtype=object)

In [22]:
# Tratar eventos graves e não graves
df_encerrados['Gravidade'] = 'Nao grave'
df_encerrados.loc[df_encerrados['Classificação de gravidade'].str.contains('Grave', na = False), 'Gravidade'] = 'Grave'

In [23]:
df_encerrados['Geral'] = 'Encerrado'

In [24]:
df_encerrados.loc[df_encerrados['Encerramento'] == 'Nao encerrado', 'Geral'] = 'Nao encerrado'

In [25]:
df_encerrados = df_encerrados[['Data da Notificação', 'Gravidade', 'Encerramento', 'Geral']]
df_encerrados

,Data da Notificação,Gravidade,Encerramento,Geral
0,09/10/2021,Nao grave,Municipal,Encerrado
1,09/10/2021,Nao grave,Municipal,Encerrado
2,09/10/2021,Nao grave,Municipal,Encerrado
3,10/10/2021,Nao grave,Municipal,Encerrado
4,10/10/2021,Nao grave,Municipal,Encerrado
...,...,...,...,...
18174,28/01/2021,Nao grave,Municipal,Encerrado
18177,28/01/2021,Nao grave,Municipal,Encerrado
18178,28/01/2021,Nao grave,Nao encerrado,Nao encerrado
18179,27/01/2021,Nao grave,Municipal,Encerrado


In [26]:
df_encerrados.loc[df_encerrados['Geral'] == 'Nao encerrado'].shape

(7792, 4)

In [27]:
df_encerrados.to_excel('df_completo.xlsx')